In [10]:
from pathlib import Path

def find_project_root(name_substring: str = "riseholme",
                      required_subpath: Path = Path("Data") / "Normal" / "alltogether",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst passt (Name + required_subpath).
      2. Prüft alle direkten Unterverzeichnisse (Geschwister), ob dort
         name_substring im Namen ist und required_subpath existiert.
    Gibt das gefundene Verzeichnis (Path) zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Ist p selbst unser Projekt-Root?
        if name_substring.lower() in p.name.lower() and (p / required_subpath).is_dir():
            return p

        # 2) Sonst durchsuche alle Unterordner von p nach unserem Projekt-Root
        for child in p.iterdir():
            if (child.is_dir()
                and name_substring.lower() in child.name.lower()
                and (child / required_subpath).is_dir()):
                return child

    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält "
                            f"und '{name_substring}' im Namen trägt (Beginn: {start})")

# Verwende die neue Funktion
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# Jetzt kannst du weitermachen:
image_dir = BASE_DIR / 'Data' / 'Normal' / 'alltogether'
label_dir = BASE_DIR / 'labels' / 'alltogether'
label_dir.mkdir(parents=True, exist_ok=True)

class_id = 0  # Klasse 0 = "ripe"
count = 0

for img_path in image_dir.iterdir():
    if img_path.suffix.lower() in {'.jpg', '.jpeg', '.png'}:
        txt_path = label_dir / f"{img_path.stem}.txt"
        with open(txt_path, 'w') as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        count += 1

print(f"✅ {count} Labels erstellt in {label_dir}")


✅ Projekt-Root gefunden: c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021
✅ 961 Labels erstellt in c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021\labels\alltogether


datensatz splitten

In [11]:
from pathlib import Path
import sys
!{sys.executable} -m pip install scikit-learn

from sklearn.model_selection import train_test_split
import shutil


def find_project_root(required_subpath: Path = Path("Data") / "Normal" / "alltogether",
                      start: Path = Path.cwd()) -> Path:
    """
    Durchläuft start und alle Eltern:
      1. Prüft, ob das aktuelle Verzeichnis selbst required_subpath enthält.
      2. Durchsucht alle Unterordner jedes Eltern-Verzeichnisses nach required_subpath.
    Gibt das erste gefundene Verzeichnis zurück oder wirft FileNotFoundError.
    """
    for p in [start] + list(start.parents):
        # 1) Passt p selbst?
        if (p / required_subpath).is_dir():
            return p
        # 2) Oder eine seiner Unterordner?
        for child in p.iterdir():
            if child.is_dir() and (child / required_subpath).is_dir():
                return child
    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält (Beginn: {start})")

# 1) Projekt-Root ermitteln
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# 2) Quell- und Ziel-Pfade definieren
IMG_SRC   = BASE_DIR / 'Data'    / 'Normal' / 'alltogether'
LBL_SRC   = BASE_DIR / 'labels'  / 'alltogether'

IMG_TRAIN = BASE_DIR / 'images'  / 'train'
IMG_VAL   = BASE_DIR / 'images'  / 'val'
LBL_TRAIN = BASE_DIR / 'labels'  / 'train'
LBL_VAL   = BASE_DIR / 'labels'  / 'val'

# 3) Zielordner anlegen
for d in (IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL):
    d.mkdir(parents=True, exist_ok=True)

# 4) Dateinamen (Stem) sammeln und splitten
files = [f.stem for f in IMG_SRC.iterdir() if f.suffix.lower() in {'.png', '.jpg', '.jpeg'}]
train, val = train_test_split(files, test_size=0.2, random_state=42)

# 5) Kopieren
for f in train:
    # Bild-Extension automatisch prüfen
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_TRAIN / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_TRAIN / src_lbl.name)

for f in val:
    src_img = next(IMG_SRC.glob(f"{f}.*"))
    shutil.copy(src_img, IMG_VAL / src_img.name)

    src_lbl = LBL_SRC / f"{f}.txt"
    shutil.copy(src_lbl, LBL_VAL / src_lbl.name)

print(f"✅ Train: {len(train)} | Val: {len(val)} Dateien kopiert")


✅ Projekt-Root gefunden: c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021
✅ Train: 768 | Val: 193 Dateien kopiert


In [ ]:
import sys
!{sys.executable} -m pip install ultralytics

from ultralytics import YOLO
model = YOLO('yolov8n.pt')             # oder yolov8s.pt, -m.pt etc.
model.train(
    data='data.yaml',                  # deine Konfig
    epochs=50,                         # je nach Bedarf , 25 reichen (1h)
    imgsz=640,
    batch=4,
    augment=True,
    project='runs/train',              # Ausgabeordner
    name='alltogether_brrr'                   # Lauf-Name
)



New https://pypi.org/project/ultralytics/8.3.127 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.109  Python-3.12.4 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=alltogether_brrr8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning C:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021\labels\train.cache... 840 images, 0 backgrounds, 0 corrupt: 100%|██████████| 840/840 [00:00<?, ?it/s]
val: Scanning C:\Users\Maxi\Documents\Forschsem\Erdbeeren\Riseholme-2021\labels\val.cache... 261 images, 0 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]


Plotting labels to runs\train\alltogether_brrr8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\train\alltogether_brrr8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      3.18G     0.2882     0.9826      1.021         10        640: 100%|██████████| 210/210 [00:30<00:00,  6.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:04<00:00,  7.94it/s]

                   all        261        261          1          1      0.995      0.975


...

                   all        261        261          1          1      0.995      0.995



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      40/50      2.53G    0.05275     0.1256     0.8885          9        640: 100%|██████████| 210/210 [00:27<00:00,  7.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:03<00:00,  9.56it/s]

                   all        261        261          1          1      0.995      0.995


Closing dataloader mosaic
